In [1]:
import torch
torch.cuda.is_available()

True

In [1]:
from benchmarl.algorithms import MappoConfig
from benchmarl.environments import VmasTask
from benchmarl.experiment import Experiment, ExperimentConfig
from benchmarl.models.mlp import MlpConfig

from ComplexVmasTask import ComplexVmasTask

In [3]:
!wandb enabled

W&B enabled.


In [ ]:
import gym
env = gym.make("CartPole-v1")
observation, info = env.reset(seed=42)

for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

c:\Users\Josh\Documents\yr4\s2\COMP4900_MultiAgent\COMP-4900-MARL-Project\venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [4]:
# Loads from "benchmarl/conf/experiment/base_experiment.yaml"
experiment_config = ExperimentConfig.get_from_yaml()

# Loads from "benchmarl/conf/task/vmas/balance.yaml"
# task = VmasTask.SIMPLE_SPEAKER_LISTENER.get_from_yaml()
task = ComplexVmasTask.SIMPLE_SPEAKER_LISTENER.get_from_yaml()
print(task)

# Loads from "benchmarl/conf/algorithm/mappo.yaml"
algorithm_config = MappoConfig.get_from_yaml()

# Loads from "benchmarl/conf/model/layers/mlp.yaml"
model_config = MlpConfig.get_from_yaml()
critic_model_config = MlpConfig.get_from_yaml()

# THIS IS VERY IMPORTANT !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
experiment_config.on_policy_collected_frames_per_batch = 6000
experiment_config.evaluation = True
experiment_config.render = True
# experiment_config.evaluation_interval = 12_000
experiment_config.evaluation_interval = 6_000
experiment_config.evaluation_episodes = 10

experiment_config.max_n_iters = 5
experiment_config.loggers = ["csv"]
experiment_config.create_json = True
experiment_config.save_folder = "experiments"
# experiment_config.train_device = "cuda"
# experiment_config.sampling_device = "cuda"

# Create the experiment
experiment = Experiment(
    task=task,
    algorithm_config=algorithm_config,
    model_config=model_config,
    critic_model_config=critic_model_config,
    seed=0,
    config=experiment_config,
)

ComplexVmasTask.SIMPLE_SPEAKER_LISTENER: (config={'max_steps': 100})
DEBUGGING device=cpu
DEBUGGING device=cpu


In [5]:
experiment.run()

mean return = -216.00315856933594: 100%|██████████| 5/5 [04:12<00:00, 50.51s/it]


# clean approach this time

In [1]:
from benchmarl.algorithms import MappoConfig, MaddpgConfig
from benchmarl.environments import VmasTask
from benchmarl.experiment import Experiment, ExperimentConfig
from benchmarl.models.mlp import MlpConfig
from ComplexVmasTask import ComplexVmasTask

# Loads from "benchmarl/conf/experiment/base_experiment.yaml"
experiment_config = ExperimentConfig.get_from_yaml()

# Loads from "benchmarl/conf/task/vmas/balance.yaml"
# task = VmasTask.SIMPLE_SPEAKER_LISTENER.get_from_yaml()
task = ComplexVmasTask.SIMPLE_SPEAKER_LISTENER.get_from_yaml()
print(task)

# Loads from "benchmarl/conf/algorithm/mappo.yaml"
algorithm_config = MappoConfig.get_from_yaml()
# algorithm_config = MaddpgConfig.get_from_yaml()

# Loads from "benchmarl/conf/model/layers/mlp.yaml"
model_config = MlpConfig.get_from_yaml()
critic_model_config = MlpConfig.get_from_yaml()

# THIS IS VERY IMPORTANT !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
experiment_config.on_policy_collected_frames_per_batch = 6000
experiment_config.evaluation = True
experiment_config.render = True
# experiment_config.evaluation_interval = 12_000
experiment_config.evaluation_interval = 6_000
experiment_config.evaluation_episodes = 10

experiment_config.max_n_iters = 1
experiment_config.loggers = ["csv"]
experiment_config.create_json = True
experiment_config.save_folder = "experiments2"

# Create the experiment
experiment = Experiment(
    task=task,
    algorithm_config=algorithm_config,
    model_config=model_config,
    critic_model_config=critic_model_config,
    seed=0,
    config=experiment_config,
)

# Log experiment folder path
print(experiment.name)

# Run experiment
experiment.run()

d:\Work\Carleton\Winter 2024\COMP 4900\project\venv\lib\site-packages\benchmarl\conf\task\vmas\simple_speaker_listener.yaml
ComplexVmasTask.SIMPLE_SPEAKER_LISTENER: (config={'max_steps': 100})
DEBUGGING cpu
DEBUGGING reached env_make_world
DEBUGGING cpu
DEBUGGING reached env_make_world
mappo_simple_speaker_listener_mlp__abd3c2c2_24_03_14-18_34_53


  0%|          | 0/1 [00:00<?, ?it/s]d:\Work\Carleton\Winter 2024\COMP 4900\project\venv\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))
mean return = -321.00897216796875: 100%|██████████| 1/1 [00:45<00:00, 45.89s/it]


In [2]:
# Convert pt files to gifs
video_file_path = f'{experiment_config.save_folder}/{experiment.name}/{experiment.name}/videos'
print(video_file_path)

experiments2/mappo_simple_speaker_listener_mlp__abd3c2c2_24_03_14-18_34_53/mappo_simple_speaker_listener_mlp__abd3c2c2_24_03_14-18_34_53/videos


In [6]:
import torch
import imageio
import os
import shutil

def convert_pt_to_gif(folder_path: str):
    """
    Convert .pt files within specified folder_path into gifs
    """
    num_videos = 1

    for i in range(num_videos):
        VID_NAME = f'{folder_path}/eval_video_{i}'

        folder = 'frames'
        os.makedirs(folder, exist_ok=True)

        tensor = torch.load(f'./{VID_NAME}.pt')
        frames = tensor.shape[1]

        for i in range(frames):
            frame = tensor[0, i].permute(1, 2, 0).numpy().astype('uint8')
            imageio.imwrite(f'./frames/frame_{i}.png', frame)

        images = []
        for i in range(frames):
            images.append(imageio.imread(f'./frames/frame_{i}.png'))

        imageio.mimsave(f'{VID_NAME}.gif', images, fps=100)
        shutil.rmtree(folder)


convert_pt_to_gif(video_file_path)

C:\Users\smohapatra\AppData\Local\Temp\ipykernel_26616\2992044582.py:27: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f'./frames/frame_{i}.png'))
